In [4]:
## DONT CHANGE THIS CELL
!wget http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip

--2021-10-25 17:13:52--  http://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Resolving www.cse.iitd.ac.in (www.cse.iitd.ac.in)... 103.27.9.152
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip [following]
--2021-10-25 17:13:52--  https://www.cse.iitd.ac.in/~mausam/courses/col772/autumn2021/A3/data.zip
Connecting to www.cse.iitd.ac.in (www.cse.iitd.ac.in)|103.27.9.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217313 (212K) [application/zip]
Saving to: ‘data.zip.1’

data.zip.1          100%[===================>] 212.22K   234KB/s    in 0.9s    

2021-10-25 17:13:53 (234 KB/s) - ‘data.zip.1’ saved [217313/217313]



In [5]:
## Import relevant packages

import os

import time
import datetime

import torch
import torch.optim as O
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import logging
from argparse import ArgumentParser

from pdb import set_trace

In [6]:
import zipfile
from torchtext.legacy.data import Field, BucketIterator, TabularDataset
import spacy
import pandas as pd
import numpy as np
import random

In [7]:
torch.manual_seed(4)
random.seed(4)
np.random.seed(4)

In [8]:
with zipfile.ZipFile("./data.zip", 'r') as zip_ref:
    zip_ref.extractall("./NLP_A3")

In [9]:
training_data_path = "./NLP_A3/data/train/train.data.txt"
validation_data_path = "./NLP_A3/data/validation/validation.data.txt"
training_data_predict_path = "./NLP_A3/data/train/train.gold.txt"
validation_data_predict_path = "./NLP_A3/data/validation/validation.gold.txt"

In [10]:
! pip install transformers

     |████████████████████████████████| 2.9 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 39.6 MB/s 
     |████████████████████████████████| 895 kB 45.4 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 3.3 MB 34.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
from transformers import TrainingArguments, Trainer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [12]:
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [13]:
from torch.utils.data import Dataset

In [14]:
class Customed_Dataset(Dataset):
  def __init__(self, X, y = None):
    self.text_encodings = X
    self.labels = y
  
  def __getitem__(self, index):
    sample = {k: torch.tensor(v[index]) for k , v in self.text_encodings.items()}
    if self.labels:
      sample['labels'] = torch.tensor(self.labels[index])

    return sample
  
  def __len__(self):
    return len(self.text_encodings['input_ids'])

In [15]:
def Dataset(tokenizer):
  file_ = pd.read_csv(training_data_path,sep='\t', header = None, names = ['word', 'POS', 'index_sent1', 'sent1', 'sent2'])
  file_v = pd.read_csv(validation_data_path,sep='\t', header = None, names = ['word', 'POS', 'index_sent1', 'sent1', 'sent2'])
  
  with open(training_data_predict_path) as f:
    data_predict = f.readlines()
    f.close()
  with open(validation_data_predict_path) as f:
    data_predict_v = f.readlines()
    f.close()
  file_predict = pd.DataFrame(data_predict, columns= ['category'])
  file_predict_v = pd.DataFrame(data_predict_v, columns= ['category'])
  for i in file_predict.index:
    idx1 = file_predict.at[i, 'category']
    if idx1[0] == 'T':
      file_predict.at[i, 'category'] = 1
    else:
      file_predict.at[i, 'category'] = 0
  for i in file_predict_v.index:
    idx1 = file_predict_v.at[i, 'category']
    if idx1[0] == 'T':
      file_predict_v.at[i, 'category'] = 1
    else:
      file_predict_v.at[i, 'category'] = 0
  file_['category'] = file_predict['category']
  file_v['category'] = file_predict_v['category']
  
  

  Y_train = list(file_['category'])
  Y_val = list(file_v['category'])

  X_train = tokenizer(list(file_['sent1']), list(file_['sent2']) ,padding = True, truncation = True)
  X_val = tokenizer(list(file_v['sent1']), list(file_v['sent2']) , padding = True, truncation = True)
  # print((X_train['input_ids'][0]))
  # print((X_val['input_ids'][0]))

  train_dataset = Customed_Dataset(X_train, Y_train)
  val_dataset = Customed_Dataset(X_val, Y_val)

  return train_dataset, val_dataset

In [16]:
train_dataset, val_dataset = Dataset(tokenizer)

In [17]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [18]:
def compute_metrics(eval_pred):
    pred, labels = eval_pred
    pred = np.argmax(pred, axis=1)
    dict_metric = {}

    dict_metric["accuracy"] = accuracy_score(y_true=labels, y_pred=pred)
    dict_metric["recall"] = recall_score(y_true=labels, y_pred=pred)
    dict_metric["precision"] = precision_score(y_true=labels, y_pred=pred)
    dict_metric["f1"] = f1_score(y_true=labels, y_pred=pred)

    return dict_metric


In [19]:
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [20]:
args = TrainingArguments(output_dir = "saved_model", evaluation_strategy = "epoch",num_train_epochs=3,save_total_limit = 1,logging_steps = 100)

In [21]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [22]:
trainer.train()

***** Running training *****
  Num examples = 5428
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2037


Epoch,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1,0.562100,0.620081,0.695925,0.768025,0.671233,0.716374
2,0.263000,1.023961,0.697492,0.758621,0.675978,0.714919
3,0.206500,1.424826,0.705329,0.796238,0.673740,0.729885


Saving model checkpoint to saved_model/checkpoint-500
Configuration saved in saved_model/checkpoint-500/config.json
Model weights saved in saved_model/checkpoint-500/pytorch_model.bin
Deleting older checkpoint [saved_model/checkpoint-2000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 638
  Batch size = 8
Saving model checkpoint to saved_model/checkpoint-1000
Configuration saved in saved_model/checkpoint-1000/config.json
Model weights saved in saved_model/checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [saved_model/checkpoint-500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 638
  Batch size = 8
Saving model checkpoint to saved_model/checkpoint-1500
Configuration saved in saved_model/checkpoint-1500/config.json
Model weights saved in saved_model/checkpoint-1500/pytorch_model.bin
Deleting older checkpoint [saved_model/checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to saved_model/checkpoint-

TrainOutput(global_step=2037, training_loss=0.36335485134864476, metrics={'train_runtime': 600.5788, 'train_samples_per_second': 27.114, 'train_steps_per_second': 3.392, 'total_flos': 594139707439920.0, 'train_loss': 0.36335485134864476, 'epoch': 3.0})

In [23]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 638
  Batch size = 8


{'epoch': 3.0,
 'eval_accuracy': 0.7053291536050157,
 'eval_f1': 0.7298850574712644,
 'eval_loss': 1.4248260259628296,
 'eval_precision': 0.6737400530503979,
 'eval_recall': 0.7962382445141066,
 'eval_runtime': 6.5297,
 'eval_samples_per_second': 97.707,
 'eval_steps_per_second': 12.252}

In [24]:
trainer.evaluate(train_dataset)

***** Running Evaluation *****
  Num examples = 5428
  Batch size = 8


{'epoch': 3.0,
 'eval_accuracy': 0.9802873986735445,
 'eval_f1': 0.9803272660415518,
 'eval_loss': 0.0935121402144432,
 'eval_precision': 0.978348623853211,
 'eval_recall': 0.9823139277818718,
 'eval_runtime': 54.7969,
 'eval_samples_per_second': 99.057,
 'eval_steps_per_second': 12.391}

In [25]:
trainer.save_model("./FINAL_MODEL/model")

Saving model checkpoint to ./FINAL_MODEL/model
Configuration saved in ./FINAL_MODEL/model/config.json
Model weights saved in ./FINAL_MODEL/model/pytorch_model.bin


In [26]:
tokenizer.save_pretrained("./FINAL_MODEL/tokenizer")

tokenizer config file saved in ./FINAL_MODEL/tokenizer/tokenizer_config.json
Special tokens file saved in ./FINAL_MODEL/tokenizer/special_tokens_map.json


('./FINAL_MODEL/tokenizer/tokenizer_config.json',
 './FINAL_MODEL/tokenizer/special_tokens_map.json',
 './FINAL_MODEL/tokenizer/vocab.txt',
 './FINAL_MODEL/tokenizer/added_tokens.json',
 './FINAL_MODEL/tokenizer/tokenizer.json')

In [27]:
!zip -r 2018EE10493_B_model.zip **

updating: data.zip (stored 0%)
updating: FINAL_MODEL/ (stored 0%)
updating: FINAL_MODEL/model/ (stored 0%)
updating: FINAL_MODEL/model/config.json (deflated 49%)
updating: FINAL_MODEL/model/pytorch_model.bin (deflated 7%)
updating: FINAL_MODEL/model/training_args.bin (deflated 48%)
updating: FINAL_MODEL/tokenizer/ (stored 0%)
updating: FINAL_MODEL/tokenizer/tokenizer_config.json (deflated 39%)
updating: FINAL_MODEL/tokenizer/special_tokens_map.json (deflated 40%)
updating: FINAL_MODEL/tokenizer/vocab.txt (deflated 49%)
updating: FINAL_MODEL/tokenizer/tokenizer.json (deflated 57%)
updating: NLP_A3/ (stored 0%)
updating: NLP_A3/data/ (stored 0%)
updating: NLP_A3/data/train/ (stored 0%)
updating: NLP_A3/data/train/train.gold.txt (deflated 90%)
updating: NLP_A3/data/train/train.data.txt (deflated 63%)
updating: NLP_A3/data/validation/ (stored 0%)
updating: NLP_A3/data/validation/validation.data.txt (deflated 58%)
updating: NLP_A3/data/validation/validation.gold.txt (deflated 87%)
updating: